In [1]:
import csv
import datetime


### 0. url-subject map 생성.
### 1. 먼저 class-central review platform 돌면서 pattern 추출,
### 2. 다음으로 coursetalk review platform.


In [4]:
def sort_by_num(x):
    if x=="id":
        return 0
    else:
        return int(x)


In [4]:
subjectMap = {}
with open("./MOOC_pattern_extraction/url_subject_info.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        subjectMap[row['url']] = row['subject']

In [5]:
## read course Data set
courseList = []
with open("./MOOC_data/xtData/(cleand)ClassCentral_course.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        courseList.append(row)
        
print(len(courseList))

3153


In [6]:
## read review Data set
reviewList = []
with open("./MOOC_data/xtData/(Date_Adjusted)ClassCentral_review_renewed.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        reviewList.append(row)
print(len(reviewList))

20903


<h3> Extract Pattern </h3>

In [7]:
## 사용자, 사용자의 수강리스트 만들기.

reviewers =[]
for review in reviewList:
    
    
    # reviewer id가 없을 경우 continue
    if review['reviewer_url'] == "null":
        continue
        
    
    # reviewers list 를 돌면서 course 추가
    flag = False
    for index, re in enumerate(reviewers):
        
        
        if re['reviewer'] == review['reviewer_url']:
            reviewers[index]['courses'].append(dict({
                    'date': review['review_date'],
                    'course': review['url'],
                    
                
                }))
            flag = True
            
            
    
    if flag == False:
        reviewers.append(dict({
            'reviewer': review['reviewer_url'],
            'courses': [dict({
                'date': review['review_date'],
                'course': review['url'],
                
            })]
            
        }))
        

In [37]:
final_t1_trans = []
final_t2_trans = []
final_t3_trans = []
final_t4_trans = []

# course directed edge list 생성
for re in reviewers:

    t1_nodes = []
    t2_nodes = []
    t3_nodes = []
    t4_nodes = []
       
    for co in re['courses']:

               
        date = datetime.datetime.strptime(co['date'], "%Y-%m-%d") 
        
        if date.year == 2013:

            # T1: 
            if date.month > 6 and date.month <= 12:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
        
        elif date.year == 2014:
            
            if date.month > 0 and date.month < 7:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T2: 
            else:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                

        elif date.year == 2015:
            
            if date.month > 0 and date.month < 7:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T3: 15.7~12
            else:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2016:
            
            if date.month > 0 and date.month < 7:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T4
            else:
                t4_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2017:
            
            if date.month > 0 and date.month < 7:
            
                t4_nodes.append(dict({
                        'course': co['course'],
                        'date': co['date']           
                    }))
    
    # 날짜별 sorting
    t1_nodes.sort(key=lambda item:item['date'], reverse=False)
    t2_nodes.sort(key=lambda item:item['date'], reverse=False)
    t3_nodes.sort(key=lambda item:item['date'], reverse=False)
    t4_nodes.sort(key=lambda item:item['date'], reverse=False)
    
    # 같은 vertex time에 있는 노드가 두개 이상인지 체크, 2개 이상일 경우 date 비교 후 edgelist에 추가
    temp_dict1 = extract_pattern(t1_nodes, re['reviewer'])
    temp_dict2 = extract_pattern(t2_nodes, re['reviewer'])
    temp_dict3 = extract_pattern(t3_nodes, re['reviewer'])
    temp_dict4 = extract_pattern(t4_nodes, re['reviewer'])
    if temp_dict1:
        final_t1_trans.append(temp_dict1)
    if temp_dict2:
        final_t2_trans.append(temp_dict2)
    if temp_dict3:
        final_t3_trans.append(temp_dict3)
    if temp_dict4:
        final_t4_trans.append(temp_dict4)
    
       


In [36]:
def transcript_to_subject_mapping(transList):
    
    for index, trans in enumerate(transList):
        
        for j, item in enumerate(trans['item-set']):
            
            if item not in subjectMap.keys():
                transList[index]['item-set'].pop(j)
                if len(transList[index]['item-set']) == 0: # pop 이후의 transcript list 길이가 0인 경우 해당 transcript 삭제
                    transList.pop(index)
            else:
                transList[index]['item-set'][j] = subjectMap[item]
                               
       
    return transList;


In [35]:
def extract_pattern(nodes, reviewer_id):
    
    tempDict = {}
    # nodes 사이즈가 0인 경우 return
    if len(nodes) == 0:
        return tempDict
#     print(reviewer_id)
    # 사이즈가 1이상인 경우
    tempDict = {
            'id': reviewer_id,
            'item-set': []
    }
    for i, node in enumerate(nodes):
#         print(node)
        tempDict['item-set'].append(node['course'])
        
    return tempDict    

In [41]:
t1_trans_final = transcript_to_subject_mapping(final_t1_trans)
t2_trans_final = transcript_to_subject_mapping(final_t2_trans)
t3_trans_final = transcript_to_subject_mapping(final_t3_trans)
t4_trans_final = transcript_to_subject_mapping(final_t4_trans)


In [43]:
# write sequence list
fieldnames = ['id', 'item-set']
with open('./MOOC_pattern_extraction/classCentral_transaction_t1.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t1_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/classCentral_transaction_t2.csv', 'w', newline='') as write_file:

    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t2_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/classCentral_transaction_t3.csv', 'w', newline='') as write_file:

    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t3_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/classCentral_transaction_t4.csv', 'w', newline='') as write_file:

    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t4_trans_final:
        dict_writer.writerow(row)

### extract pattern at courseTalk

In [44]:
# data load

courseList = []
with open("./MOOC_data/xtData/(cleand_added_missing)Coursetalk_course.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['url'] = row['url'].replace("/enroll?course_id=", "")
        row['url'] = row['url'].replace("&direct_enroll=true", "")
           
        courseList.append(row)
        
print(len(courseList))

## read review Data set
reviewList = []
with open("./MOOC_data/xtData/CourseTalk_review_renewed.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['review_value'] = int(row['review_value'])/2 # 0~10 scale이므로 2로 나누어 0~5 scale로 변환.
        reviewList.append(row)
print(len(reviewList))

3331
23604


In [45]:
## 사용자, 사용자의 수강리스트 만들기.

reviewers =[]
for review in reviewList:
    
    
    # reviewer id가 없을 경우 continue
    if review['reviewer_url'] == "null":
        continue
        
    
    # reviewers list 를 돌면서 course 추가
    flag = False
    for index, re in enumerate(reviewers):
                
        if re['reviewer'] == review['reviewer_url']:
            reviewers[index]['courses'].append(dict({
                    'date': review['review_date'],
                    'course': review['course_id']
                }))
            flag = True
            
            
    
    if flag == False:
        reviewers.append(dict({
            'reviewer': review['reviewer_url'],
            'courses': [dict({
                'date': review['review_date'],
                'course': review['course_id']
            })]
            
        }))
        

In [46]:
final_t1_trans = []
final_t2_trans = []
final_t3_trans = []
final_t4_trans = []

# course directed edge list 생성
for re in reviewers:

    t1_nodes = []
    t2_nodes = []
    t3_nodes = []
    t4_nodes = []
       
    for co in re['courses']:

               
        date = datetime.datetime.strptime(co['date'], "%Y-%m-%d") 
        
        if date.year == 2013:

            # T1: 
            if date.month > 6 and date.month <= 12:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
        
        elif date.year == 2014:
            
            if date.month > 0 and date.month < 7:
                t1_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T2: 
            else:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                

        elif date.year == 2015:
            
            if date.month > 0 and date.month < 7:
                t2_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T3: 15.7~12
            else:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2016:
            
            if date.month > 0 and date.month < 7:
                t3_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
            # T4
            else:
                t4_nodes.append(dict({
                    'course': co['course'],
                    'date': co['date']           
                }))
                
        elif date.year == 2017:
            
            if date.month > 0 and date.month < 7:
            
                t4_nodes.append(dict({
                        'course': co['course'],
                        'date': co['date']           
                    }))
    
    # 날짜별 sorting
    t1_nodes.sort(key=lambda item:item['date'], reverse=False)
    t2_nodes.sort(key=lambda item:item['date'], reverse=False)
    t3_nodes.sort(key=lambda item:item['date'], reverse=False)
    t4_nodes.sort(key=lambda item:item['date'], reverse=False)
    
    # 같은 vertex time에 있는 노드가 두개 이상인지 체크, 2개 이상일 경우 date 비교 후 edgelist에 추가
    temp_dict1 = extract_pattern(t1_nodes, re['reviewer'])
    temp_dict2 = extract_pattern(t2_nodes, re['reviewer'])
    temp_dict3 = extract_pattern(t3_nodes, re['reviewer'])
    temp_dict4 = extract_pattern(t4_nodes, re['reviewer'])
    if temp_dict1:
        final_t1_trans.append(temp_dict1)
    if temp_dict2:
        final_t2_trans.append(temp_dict2)
    if temp_dict3:
        final_t3_trans.append(temp_dict3)
    if temp_dict4:
        final_t4_trans.append(temp_dict4)
    
       


In [47]:
t1_trans_final_courseTalk = transcript_to_subject_mapping(final_t1_trans)
t2_trans_final_courseTalk = transcript_to_subject_mapping(final_t2_trans)
t3_trans_final_courseTalk = transcript_to_subject_mapping(final_t3_trans)
t4_trans_final_courseTalk = transcript_to_subject_mapping(final_t4_trans)


In [48]:
# write sequence list
fieldnames = ['id', 'item-set']
        
with open('./MOOC_pattern_extraction/courseTalk_transaction_t1.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t1_trans_final_courseTalk:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_transaction_t2.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t2_trans_final_courseTalk:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_transaction_t3.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t3_trans_final_courseTalk:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_transaction_t4.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t4_trans_final_courseTalk:
        dict_writer.writerow(row)

### Integrate transaction data

In [49]:
t1_trans_final.extend(t1_trans_final_courseTalk)
t2_trans_final.extend(t2_trans_final_courseTalk)
t3_trans_final.extend(t3_trans_final_courseTalk)
t4_trans_final.extend(t4_trans_final_courseTalk)


In [50]:
# write sequence list
fieldnames = ['id', 'item-set']
with open('./MOOC_pattern_extraction/integrated_transaction_t1.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t1_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_transaction_t2.csv', 'w', newline='') as write_file:
    
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t2_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_transaction_t3.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t3_trans_final:
        dict_writer.writerow(row)
        
# write sequence list
with open('./MOOC_pattern_extraction/courseTalk_transaction_t4.csv', 'w', newline='') as write_file:
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    for row in t4_trans_final:
        dict_writer.writerow(row)

# Extract frequency of each transaction

### 먼저 각 item-set string 정렬

In [53]:
def itemset_sort(trans_list):
    for index, tr in enumerate(trans_list):
        trans_list[index]['item-set'].sort()
        
    return trans_list

In [54]:
t1_trans_sorted = itemset_sort(t1_trans_final)

In [55]:
t1_trans_sorted[0]

{'id': '/@dhawal',
 'item-set': ['Algorithms',
  'Business Strategy',
  'Data Analysis',
  'Economics',
  'Software Development']}

In [56]:
['he','she'] == ['he', 'she']

True